In [11]:
# In case with camera data we have to iterate back with 10 minute intervals from the current timestamp

import requests
import time
import json
import os

req_header = {"apiToken":"f8966829-616a-4e45-aa26-ab2de0d42515"}

In [12]:
# get the camera list

url = " https://infraweb-rws.fi/infralink/cams/camlist"
response = requests.get(url, headers=req_header)
response.json()

[{'name': 'VT4 Jyväskylä Vaajakoski',
  'lat': 61.99935656434613,
  'lon': 25.500450069798536,
  'variants': [{'name': 'Jyväskylään', 'id': 'C0951002'}]},
 {'name': 'VT9 Jyväskylä Pumperi',
  'lat': 62.29026582544774,
  'lon': 25.72040656942994,
  'variants': [{'name': 'Jyäskylään', 'id': 'C0951101'},
   {'name': 'Jämsään', 'id': 'C0951102'},
   {'name': 'Keljonkankaantie', 'id': 'C0951103'},
   {'name': 'Maisema', 'id': 'C0951106'},
   {'name': 'Tienpinta', 'id': 'C0951109'}]},
 {'name': 'VT4 Jyväskylä Puuppola',
  'lat': 62.341198537154966,
  'lon': 25.70431603797372,
  'variants': [{'name': 'Ouluun', 'id': 'C0951501'},
   {'name': 'Jyäskylään', 'id': 'C0951502'},
   {'name': 'Tienpinta', 'id': 'C0951509'}]},
 {'name': 'VT4 Jyväskylä Palokka',
  'lat': 62.21160305771549,
  'lon': 25.720218341690536,
  'variants': [{'name': 'Ouluun', 'id': 'C0952001'},
   {'name': 'Jyäskylään', 'id': 'C0952002'},
   {'name': 'Palokka', 'id': 'C0952003'},
   {'name': 'Laajavuori', 'id': 'C0952004'},
  

In [13]:
# test camera index 0
camera_index = 0

cam_data = response.json()[camera_index]
cam_data

{'name': 'VT4 Jyväskylä Vaajakoski',
 'lat': 61.99935656434613,
 'lon': 25.500450069798536,
 'variants': [{'name': 'Jyväskylään', 'id': 'C0951002'}]}

In [14]:
# we can have multiple variants depending on the direction, let's only test one

cam_id = cam_data['variants'][0]['id']

In [15]:
# get the latest image list for the timestamp
url = f"https://infraweb-rws.fi/infralink/cams/imglist/?id={cam_id}"

response = requests.get(url, headers=req_header)
response.json()

[['2025-03-31T08:54:55.513868', 1743400495000],
 ['2025-03-31T07:54:55.513868', 1743396895000],
 ['2025-03-31T06:54:55.513868', 1743393295000],
 ['2025-03-31T05:54:55.513868', 1743389695000],
 ['2025-03-31T04:54:55.513868', 1743386095000],
 ['2025-03-31T03:54:55.513868', 1743382495000],
 ['2025-03-31T02:54:55.513868', 1743378895000],
 ['2025-03-31T01:54:55.513868', 1743375295000],
 ['2025-03-31T00:54:55.513868', 1743371695000],
 ['2025-03-30T23:54:55.513868', 1743368095000]]

In [16]:
latest_ts = response.json()[-1][1]
latest_ts

1743368095000

In [31]:
interval = 600000 # 10 minutes
# for a month worth of data
num_timestamps = 30 * 24 * 60 * 60 * 1000 // interval

timestamps = [latest_ts - i * interval for i in range(0, num_timestamps)]


In [32]:
# make directory for the images if it does not exist
path = f'images/{cam_id}'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
# we can use batch requests to speed up the process


In [33]:
from concurrent.futures import ThreadPoolExecutor

In [34]:
# iterate through the timestamps and download the images

url = f"https://infraweb-rws.fi/infralink/cams/img?id={cam_id}&ts="

urls = [url + str(ts) for ts in timestamps]

        

In [30]:
batch_size = 100

test_urls = urls[:batch_size]
# test the first 10 urls

def fetch_image(url):
    response = requests.get(url, headers=req_header)
    if response.status_code == 200:
        return response.content
    else:
        return None
    
# download the images
with ThreadPoolExecutor(max_workers=batch_size) as executor:
    responses = list(executor.map(fetch_image, test_urls))

# save the images
for image, url in zip(responses, test_urls):
    if image is not None:
        # get the timestamp from the url
        ts = url.split('=')[-1]
        # convert the timestamp to a readable format
        ts = time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime(int(ts) / 1000))
        # save the image
        with open(f'{path}/{ts}.jpg', 'wb') as f:
            f.write(image)

In [35]:
# full loop
for i in range(0, len(urls), batch_size):
    # get the batch of urls
    batch_urls = urls[i:i + batch_size]
    # download the images
    with ThreadPoolExecutor(max_workers=batch_size) as executor:
        responses = list(executor.map(fetch_image, batch_urls))
    # save the images
    for image, url in zip(responses, batch_urls):
        if image is not None:
            # get the timestamp from the url
            ts = url.split('=')[-1]
            # convert the timestamp to a readable format
            ts = time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime(int(ts) / 1000))
            # save the image
            with open(f'{path}/{ts}.jpg', 'wb') as f:
                f.write(image)